In [ ]:
import pandas as pd
import os
import torch
import numpy as np
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import torch.nn as nn
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import torch.optim
import seaborn as sns
import plotly.graph_objs as go
from plotly.offline import iplot

In [ ]:
from google.colab import files
uploaded=files.upload()

Saving monthly_hourly_load_values_2022.csv to monthly_hourly_load_values_2022.csv
Saving monthly_hourly_load_values_2023.csv to monthly_hourly_load_values_2023.csv
Saving monthly_hourly_load_values_2024.csv to monthly_hourly_load_values_2024.csv


In [ ]:
from google.colab import files
uploaded=files.upload()

Saving monthly_hourly_load_values_2024.csv to monthly_hourly_load_values_2024.csv


In [ ]:
def mark_time_features(df):
  df['day'] =   df['DateCET/CEST'].dt.day
  df['hour'] =   df['DateCET/CEST'].dt.hour
  df['month'] =   df['DateCET/CEST'].dt.month
  df['year'] =   df['DateCET/CEST'].dt.year
  return df

In [ ]:
def split_datset(df):
  df_train=df[df['year']==2023]
  df_test=df[df['year']==2024]
  return df_train, df_test

In [ ]:
def check_before_merge(df1, df2, key):
    """
    Check two DataFrames before merging on a given key column.
    """

    # Dtypes
    print("Column dtypes:")
    print(f"df1[{key}]: {df1[key].dtype}")
    print(f"df2[{key}]: {df2[key].dtype}\n")

    # Duplicates
    print("Duplicate counts in key column:")
    print(f"df1: {df1[key].duplicated().sum()} duplicates")
    print(f"df2: {df2[key].duplicated().sum()} duplicates\n")

    # Min/max ranges
    print("Key column ranges:")
    print(f"df1: {df1[key].min()} → {df1[key].max()}")
    print(f"df2: {df2[key].min()} → {df2[key].max()}\n")

    # Intersection of keys
    common = df1[key].isin(df2[key]).sum()
    print("Common keys:")
    print(f"{common} common values ({common/len(df1)*100:.2f}% of df1)\n")

    # Timezone check (only if datetime)
    if "datetime64" in str(df1[key].dtype) and "datetime64" in str(df2[key].dtype):
        tz1 = getattr(df1[key].dt, "tz", None)
        tz2 = getattr(df2[key].dt, "tz", None)
        print("Timezone info:")
        print(f"df1 timezone: {tz1}")
        print(f"df2 timezone: {tz2}\n")

In [ ]:
df_train_full = pd.DataFrame()
df_test_full = pd.DataFrame()

##Preprocessing Energy Load Dataset

In [ ]:
def pretransformation_load_dataset(df):
  df=df[df['CountryCode']=='PL']
  df=df.rename(columns={'Value':'load[MW]'})
  df=df[['DateUTC', 'load[MW]']]
  return df

In [ ]:
##---transformation Load Dataset---
#load dataset
df_load22=pd.read_csv('monthly_hourly_load_values_2022.csv', sep=';')
df_load23=pd.read_csv('monthly_hourly_load_values_2023.csv', sep='\t')
df_load24=pd.read_csv('monthly_hourly_load_values_2024.csv', sep='\t')

In [ ]:
df_load22 = pretransformation_load_dataset(df_load22)
df_load23 = pretransformation_load_dataset(df_load23)
df_load24 = pretransformation_load_dataset(df_load24)

In [ ]:
#convert to datetime [2022]
df_load22['DateUTC'] = pd.to_datetime(df_load22['DateUTC'], format="%d/%m/%Y %H:%M")

#convert to datetime [2023]
df_load23['DateUTC'] = pd.to_datetime(df_load23['DateUTC'], format="%d-%m-%Y %H:%M")

#convert to CET/CEST datetime [2024]
df_load24['DateUTC'] = pd.to_datetime(df_load24['DateUTC'], format="%d-%m-%Y %H:%M")

In [ ]:
df_load_full = pd.concat([df_load22, df_load23, df_load24])

In [ ]:
df_load_full['DateCET/CEST'] = df_load_full['DateUTC'].dt.tz_localize('UTC').dt.tz_convert('Europe/Warsaw')

In [ ]:
#extract hour, day, month, year
df_load_full = mark_time_features(df_load_full)

In [ ]:
#split dataset
df_train_load, df_test_load = split_datset(df_load_full)

In [ ]:
df_train_load.columns

Index(['DateUTC', 'load[MW]', 'DateCET/CEST', 'day', 'hour', 'month', 'year'], dtype='object')

In [ ]:
df_train_load = df_train_load[['load[MW]', 'DateCET/CEST']]
df_test_load = df_test_load[['load[MW]', 'DateCET/CEST']]

In [ ]:
df_train_load[df_train_load['DateCET/CEST'].duplicated()==True]

,load[MW],DateCET/CEST
243802,13795.2,2023-03-26 05:00:00+02:00


In [ ]:
mask = df_train_load['DateCET/CEST'] == pd.Timestamp('2023-03-26 05:00:00+02:00')
idx = df_train_load[mask].index[0]  # first matching index
df_train_load.loc[idx, 'DateCET/CEST'] = pd.Timestamp('2023-03-26 04:00:00+02:00')

In [ ]:
df_train_load[df_train_load['DateCET/CEST'].duplicated()==True]

,load[MW],DateCET/CEST


In [ ]:
df_test_load[df_test_load['DateCET/CEST'].duplicated()==True]

,load[MW],DateCET/CEST
245937,12122.61,2024-03-31 05:00:00+02:00


In [ ]:
mask = df_test_load['DateCET/CEST'] == pd.Timestamp('2024-03-31 05:00:00+02:00')
idx = df_test_load[mask].index[0]  # first matching index
df_test_load.loc[idx, 'DateCET/CEST'] = pd.Timestamp('2024-03-31 04:00:00+02:00')

In [ ]:
df_test_load[df_test_load['DateCET/CEST'].duplicated()==True]

,load[MW],DateCET/CEST


In [ ]:
#integration with train dataset
df_train_full1 = df_train_load

#intergration with test dataset
df_test_full1 = df_test_load

##Preprocessing Energy Price Dataset

In [ ]:
from google.colab import files
uploaded=files.upload()

Saving Poland_energy_price.csv to Poland_energy_price.csv


In [ ]:
#load
df_price=pd.read_csv('Poland_energy_price.csv')

In [ ]:
#convert to datetime
df_price['DateCET/CEST']= pd.to_datetime(df_price['Datetime (Local)'])
df_price['DateCET/CEST'] = df_price['DateCET/CEST'].dt.tz_localize('Europe/Warsaw', ambiguous='infer')

In [ ]:
#extract features of datetime
df_price = mark_time_features(df_price)

#rename columns
df_price = df_price.rename(columns={'Price (EUR/MWhe)':'price_energy[EUR/MWh]'})

In [ ]:
#extract features of datetime
df_price = mark_time_features(df_price)

In [ ]:
#split dataset accoring to year
df_train_price, df_test_price = split_datset(df_price)

In [ ]:
df_train_price[df_train_price['DateCET/CEST'].duplicated()==True]

,Country,ISO3 Code,Datetime (UTC),Datetime (Local),price_energy[EUR/MWh],DateCET/CEST,day,hour,month,year


In [ ]:
#select columns
df_train_price=df_train_price[['DateCET/CEST','price_energy[EUR/MWh]']]
df_test_price=df_test_price[['DateCET/CEST','price_energy[EUR/MWh]']]

In [ ]:
check_before_merge(df_train_full1, df_train_price, 'DateCET/CEST')

Column dtypes:
df1[DateCET/CEST]: datetime64[ns, Europe/Warsaw]
df2[DateCET/CEST]: datetime64[ns, Europe/Warsaw]

Duplicate counts in key column:
df1: 0 duplicates
df2: 0 duplicates

Key column ranges:
df1: 2023-01-01 00:00:00+01:00 → 2023-12-31 23:00:00+01:00
df2: 2023-01-01 00:00:00+01:00 → 2023-12-31 23:00:00+01:00

Common keys:
8760 common values (100.00% of df1)

Timezone info:
df1 timezone: Europe/Warsaw
df2 timezone: Europe/Warsaw



In [ ]:
check_before_merge(df_test_full1, df_test_price, 'DateCET/CEST')

Column dtypes:
df1[DateCET/CEST]: datetime64[ns, Europe/Warsaw]
df2[DateCET/CEST]: datetime64[ns, Europe/Warsaw]

Duplicate counts in key column:
df1: 0 duplicates
df2: 0 duplicates

Key column ranges:
df1: 2024-01-01 00:00:00+01:00 → 2024-12-31 23:00:00+01:00
df2: 2024-01-01 00:00:00+01:00 → 2024-12-31 23:00:00+01:00

Common keys:
8784 common values (100.00% of df1)

Timezone info:
df1 timezone: Europe/Warsaw
df2 timezone: Europe/Warsaw



In [ ]:
#integration with train dataset
df_train_full2= pd.merge(df_train_full1, df_train_price, on = 'DateCET/CEST', how='outer', suffixes=('_1', '_2'))

#integration with test dataset
df_test_full2= pd.merge(df_test_full1, df_test_price, on = 'DateCET/CEST', how='outer', suffixes=('_1', '_2'))

##Preprocessing PV and Wind Generation Dataset

In [ ]:
from google.colab import files
uploaded=files.upload()

Saving pse_wind_pv_generation.csv to pse_wind_pv_generation.csv


In [ ]:
#load
df_pv=pd.read_csv('pse_wind_pv_generation.csv')

In [ ]:
#insertion the missing row (two o'clock row of 29th October 2023)
missing_row = pd.DataFrame([{
    'index': '2023-10-29 02:00:00',
    'Photovoltaics generation [MWh]': 0.0,
    'Wind generation [MWh]': 1468.838	,
    'DateCET/CEST': '2023-10-29 02:00:00',
    'day': 29,
    'hour': 2,
    'month': 10,
    'year': 2023
}])

df_pv = (pd.concat([df_pv, missing_row], ignore_index=True))

In [ ]:
#convert to datetime
df_pv['DateCET/CEST']= pd.to_datetime(df_pv['index'])

In [ ]:
df_pv = df_pv.sort_values('DateCET/CEST').reset_index(drop=True)

In [ ]:
#convert to datetime
df_pv['DateCET/CEST']= pd.to_datetime(df_pv['index'])

In [ ]:
#extract features of datetime
df_pv = mark_time_features(df_pv)

In [ ]:
# #split data according to year
df_train_pv, df_test_pv = split_datset(df_pv)

In [ ]:
# Create a mask of ambiguous times (only 2021-10-31 02:00 rows are True)
ambiguous_mask = df_train_pv['DateCET/CEST'].dt.strftime("%Y-%m-%d %H:%M:%S") == "2023-10-29 02:00:00"
# Provide a list of True/False (first True = DST, second False = standard)
ambiguous_flags = ambiguous_mask.cumsum() % 2 == 1

df_train_pv['DateCET/CEST'] = df_train_pv['DateCET/CEST'].dt.tz_localize(
    "Europe/Warsaw",
    ambiguous=ambiguous_flags
)

/tmp/ipython-input-4057550635.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
df_test_pv['DateCET/CEST'] = df_test_pv['DateCET/CEST'].dt.tz_localize("Europe/Warsaw")

/tmp/ipython-input-887561454.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
check_before_merge(df_train_full2, df_train_pv, 'DateCET/CEST')

Column dtypes:
df1[DateCET/CEST]: datetime64[ns, Europe/Warsaw]
df2[DateCET/CEST]: datetime64[ns, Europe/Warsaw]

Duplicate counts in key column:
df1: 0 duplicates
df2: 0 duplicates

Key column ranges:
df1: 2023-01-01 00:00:00+01:00 → 2023-12-31 23:00:00+01:00
df2: 2023-01-01 00:00:00+01:00 → 2023-12-31 23:00:00+01:00

Common keys:
8760 common values (100.00% of df1)

Timezone info:
df1 timezone: Europe/Warsaw
df2 timezone: Europe/Warsaw



In [ ]:
check_before_merge(df_test_full2, df_test_pv, 'DateCET/CEST')

Column dtypes:
df1[DateCET/CEST]: datetime64[ns, Europe/Warsaw]
df2[DateCET/CEST]: datetime64[ns, Europe/Warsaw]

Duplicate counts in key column:
df1: 0 duplicates
df2: 0 duplicates

Key column ranges:
df1: 2024-01-01 00:00:00+01:00 → 2024-12-31 23:00:00+01:00
df2: 2024-01-01 00:00:00+01:00 → 2024-06-14 00:00:00+02:00

Common keys:
3957 common values (45.05% of df1)

Timezone info:
df1 timezone: Europe/Warsaw
df2 timezone: Europe/Warsaw



In [ ]:
#integration with train dataset
df_train_full3 = pd.merge(df_train_full2, df_train_pv, on='DateCET/CEST')

#integration with test dataset
df_test_full3 = pd.merge(df_test_full2, df_test_pv, on='DateCET/CEST')

##Preprocessing Gas Price Dataset

In [ ]:
from google.colab import files
uploaded=files.upload()

Saving prices_gas_day_ahead.csv to prices_gas_day_ahead.csv


In [ ]:
#load
df_gas = pd.read_csv('prices_gas_day_ahead.csv')

In [ ]:
df_gas.columns

Index(['date', 'indeks', 'price', 'volume'], dtype='object')

In [ ]:
#convert to datetime
df_gas['DateCET/CEST'] = pd.to_datetime(df_gas['date'], dayfirst=True)

In [ ]:
#extract features of datetime
df_gas = mark_time_features(df_gas)

#convert PLN to EUR
df_gas['price'] = df_gas['price'].map(lambda x: x/4.25)

#rename column
df_gas = df_gas.rename(columns={'price': 'price_gas[EUR/m3]'})

In [ ]:
#split data according to year
df_train_gas, df_test_gas = split_datset(df_gas)

In [ ]:
df_train_gas['DateCET/CEST'] = df_train_gas['DateCET/CEST'].dt.tz_localize('Europe/Warsaw', ambiguous='infer')
df_test_gas['DateCET/CEST'] = df_test_gas['DateCET/CEST'].dt.tz_localize('Europe/Warsaw', ambiguous='infer')

/tmp/ipython-input-1546836472.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipython-input-1546836472.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
check_before_merge(df_train_full3, df_train_gas, 'DateCET/CEST')

Column dtypes:
df1[DateCET/CEST]: datetime64[ns, Europe/Warsaw]
df2[DateCET/CEST]: datetime64[ns, Europe/Warsaw]

Duplicate counts in key column:
df1: 0 duplicates
df2: 0 duplicates

Key column ranges:
df1: 2023-01-01 00:00:00+01:00 → 2023-12-31 23:00:00+01:00
df2: 2023-01-01 00:00:00+01:00 → 2023-12-31 00:00:00+01:00

Common keys:
365 common values (4.17% of df1)

Timezone info:
df1 timezone: Europe/Warsaw
df2 timezone: Europe/Warsaw



In [ ]:
check_before_merge(df_test_full3, df_test_gas, 'DateCET/CEST')

Column dtypes:
df1[DateCET/CEST]: datetime64[ns, Europe/Warsaw]
df2[DateCET/CEST]: datetime64[ns, Europe/Warsaw]

Duplicate counts in key column:
df1: 0 duplicates
df2: 0 duplicates

Key column ranges:
df1: 2024-01-01 00:00:00+01:00 → 2024-06-14 00:00:00+02:00
df2: 2024-01-01 00:00:00+01:00 → 2024-12-31 00:00:00+01:00

Common keys:
165 common values (4.17% of df1)

Timezone info:
df1 timezone: Europe/Warsaw
df2 timezone: Europe/Warsaw



In [ ]:
#integration with train data
df_train_full4 = df_train_full3.merge(
    df_train_gas[['DateCET/CEST', 'price_gas[EUR/m3]']],
    on='DateCET/CEST',
    how='left'
)

df_train_full4['price_gas[EUR/m3]'] =  df_train_full4['price_gas[EUR/m3]'].ffill()

In [ ]:
#integration with test data
df_test_full4 = df_test_full3.merge(
    df_test_gas[['DateCET/CEST', 'price_gas[EUR/m3]']],
    on='DateCET/CEST',
    how='left'
)

df_test_full4['price_gas[EUR/m3]'] =  df_test_full4['price_gas[EUR/m3]'].ffill()

##Preprocessing Carbon Permits Price Dataset

In [ ]:
from google.colab import files
uploaded=files.upload()

Saving prices_eu_ets_all.csv to prices_eu_ets_all (1).csv


In [ ]:
#load
df_carbon = pd.read_csv('prices_eu_ets_all.csv')

In [ ]:
df_carbon.columns

Index(['date', 'indeks', 'price', 'volume'], dtype='object')

In [ ]:
#convert to datetime
df_carbon['DateCET/CEST'] = pd.to_datetime(df_carbon['date'], dayfirst = True)
df_carbon['DateCET/CEST'] = df_carbon['DateCET/CEST'].dt.tz_localize('Europe/Warsaw', ambiguous='infer')

In [ ]:
#extract features of datetime
df_carbon = mark_time_features(df_carbon)

In [ ]:
df_carbon = df_carbon.rename(columns={'price':'price_carbon_permits[EUR]'})

In [ ]:
#split data according to year
df_train_carbon, df_test_carbon = split_datset(df_carbon)

In [ ]:
df_train_carbon = df_train_carbon.sort_values('DateCET/CEST')

In [ ]:
#---filling missing price in df_train_carbon---
df_2022 = df_carbon[df_carbon['year']==2022]
missing_row_carbon = df_2022.iloc[[-1]]
df_train_carbon = (pd.concat([df_train_carbon, missing_row_carbon], ignore_index=True)).sort_values('DateCET/CEST')
df_train_carbon.loc[df_train_carbon['DateCET/CEST']=='2022-12-19 00:00:00+01:00', 'DateCET/CEST'] = '2023-01-01 00:00:00+01:00'

In [ ]:
#---filling missing price in df_test_carbon---
missing_row_carbon = df_train_carbon.iloc[[-1]]
#date = missing_row_carbon['DateCET/CEST'].values[0]
df_test_carbon = (pd.concat([df_test_carbon, missing_row_carbon], ignore_index=True)).sort_values('DateCET/CEST')
df_test_carbon.loc[df_test_carbon['DateCET/CEST']=='2023-12-18 00:00:00+01:00', 'DateCET/CEST'] = '2024-01-01 00:00:00+01:00'

In [ ]:
#extract features of datetime
df_train_carbon = mark_time_features(df_train_carbon)
df_test_carbon = mark_time_features(df_test_carbon)

In [ ]:
#----intergration with df_train_full----
df_tmp = df_train_carbon[['day', 'month', 'year', 'price_carbon_permits[EUR]']]
df_train_full5 = df_train_full4.merge(df_tmp,
                          on = ['day', 'month', 'year'],
                          how = 'left')


#update the column "price_carbon_permits"
df_train_full5['price_carbon_permits[EUR]'] = df_train_full5['price_carbon_permits[EUR]'].ffill()


In [ ]:
#----intergration with df_test_full----
#select relevant columns
df_tmp = df_test_carbon[['day', 'month', 'year', 'price_carbon_permits[EUR]']]
df_test_full5 = df_test_full4.merge(df_tmp,
                          on = ['day', 'month', 'year'],
                          how = 'left')

#update the column "price_carbon_permits"
df_test_full5['price_carbon_permits[EUR]'] = df_test_full5['price_carbon_permits[EUR]'].ffill()

##Preprocessing Coal Dataset

In [ ]:
from google.colab import files
uploaded=files.upload()

Saving PSCMI_1_2023_2025.csv to PSCMI_1_2023_2025.csv


In [ ]:
#load
df_coal = pd.read_csv('PSCMI_1_2023_2025.csv', sep=';')

In [ ]:
df_coal.columns

Index(['-', '01.2011', '02.2011', '03.2011', '04.2011', '05.2011', '06.2011',
       '07.2011', '08.2011', '09.2011',
       ...
       '09.2024', '10.2024', '11.2024', '12.2024', '01.2025', '02.2025',
       '03.2025', '04.2025', '05.2025', '06.2025'],
      dtype='object', length=175)

In [ ]:
#transposition
df_coal = df_coal.iloc[1].reset_index()

df_coal = df_coal.iloc[1:]

#column names
df_coal.columns = ["Date", "price_coal"]

In [ ]:
# Convert do datetime
df_coal["DateCET/CEST"] = pd.to_datetime(df_coal['Date'], format="%m.%Y")
df_coal['DateCET/CEST'] = df_coal['DateCET/CEST'].dt.tz_localize('Europe/Warsaw', ambiguous='infer')

In [ ]:
#mark months, years
df_coal = mark_time_features(df_coal)

In [ ]:
#conversion data [GW->KW]
df_coal['price_coal'] = df_coal['price_coal'].apply(lambda x: x/3.6)

#conversion data [PLN->EUR]
df_coal['price_coal']=df_coal['price_coal'].apply(lambda x: x*4.6)

In [ ]:
#split data according to year
df_train_coal, df_test_coal = split_datset(df_coal)

In [ ]:
#dict: key - Month, value - price_coal
train_coal_dict = dict(zip(df_train_coal['month'], df_train_coal['price_coal']))

test_coal_dict = dict(zip(df_test_coal['month'], df_test_coal['price_coal']))

In [ ]:
#----integration with train dataset----
df_train_full6 = df_train_full5.copy()

#convert train_coal_dict keys into month numbers
mapping = {int(k): v for k, v in train_coal_dict.items()}

#coal price to train dataset
df_train_full6["price_coal"] = df_train_full6["month"].map(mapping)


In [ ]:
#----integration with test dataset----
df_test_full6 = df_test_full5.copy()

#convert train_coal_dict keys into month numbers
mapping = {int(k): v for k, v in test_coal_dict.items()}

#coal price to test dataset
df_test_full6["price_coal"] = df_test_full6["month"].map(mapping)

##Preprocessing Wheather Dataset

In [ ]:
from google.colab import files
uploaded=files.upload()

Saving open-meteo-51.92N19.12E138m_22_23_24.csv to open-meteo-51.92N19.12E138m_22_23_24.csv


In [ ]:
#load
df_meteo = pd.read_csv('open-meteo-51.92N19.12E138m_22_23_24.csv', encoding='latin-1', sep=';')

In [ ]:
#convert to datetime
df_meteo['DateUTC'] = pd.to_datetime(df_meteo['Date (GTC)'])
df_meteo['DateCET/CEST'] = df_meteo['DateUTC'].dt.tz_localize('UTC').dt.tz_convert('Europe/Warsaw')

In [ ]:
#extract features of datetime
df_meteo = mark_time_features(df_meteo)

In [ ]:
#split data according to year
df_train_meteo, df_test_meteo = split_datset(df_meteo)

In [ ]:
df_train_meteo = df_train_meteo[['DateCET/CEST', 'temperature_2m (°C)', 'rain (mm)', 'wind_speed_100m (km/h)',
       'diffuse_radiation_instant (W/m?)']]

df_test_meteo = df_test_meteo[['DateCET/CEST', 'temperature_2m (°C)', 'rain (mm)', 'wind_speed_100m (km/h)',
       'diffuse_radiation_instant (W/m?)']]

In [ ]:
check_before_merge(df_train_full6, df_train_meteo, 'DateCET/CEST')

Column dtypes:
df1[DateCET/CEST]: datetime64[ns, Europe/Warsaw]
df2[DateCET/CEST]: datetime64[ns, Europe/Warsaw]

Duplicate counts in key column:
df1: 0 duplicates
df2: 0 duplicates

Key column ranges:
df1: 2023-01-01 00:00:00+01:00 → 2023-12-31 23:00:00+01:00
df2: 2023-01-01 00:00:00+01:00 → 2023-12-31 23:00:00+01:00

Common keys:
8760 common values (100.00% of df1)

Timezone info:
df1 timezone: Europe/Warsaw
df2 timezone: Europe/Warsaw



In [ ]:
check_before_merge(df_test_full6, df_test_meteo, 'DateCET/CEST')

Column dtypes:
df1[DateCET/CEST]: datetime64[ns, Europe/Warsaw]
df2[DateCET/CEST]: datetime64[ns, Europe/Warsaw]

Duplicate counts in key column:
df1: 0 duplicates
df2: 0 duplicates

Key column ranges:
df1: 2024-01-01 00:00:00+01:00 → 2024-06-14 00:00:00+02:00
df2: 2024-01-01 00:00:00+01:00 → 2024-12-31 23:00:00+01:00

Common keys:
3957 common values (100.00% of df1)

Timezone info:
df1 timezone: Europe/Warsaw
df2 timezone: Europe/Warsaw



In [ ]:
#integration with train datatset
df_train_full7 = pd.merge(df_train_full6, df_train_meteo, on='DateCET/CEST')

#integration with test datatset
df_test_full7 = pd.merge(df_test_full6, df_test_meteo, on='DateCET/CEST')

###Save datasets

In [ ]:
df_train_full7.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 17 columns):
 #   Column                            Non-Null Count  Dtype                        
---  ------                            --------------  -----                        
 0   load[MW]                          8760 non-null   float64                      
 1   DateCET/CEST                      8760 non-null   datetime64[ns, Europe/Warsaw]
 2   price_energy[EUR/MWh]             8760 non-null   float64                      
 3   index                             8760 non-null   object                       
 4   Photovoltaics generation [MWh]    8760 non-null   float64                      
 5   Wind generation [MWh]             8760 non-null   float64                      
 6   day                               8760 non-null   int32                        
 7   hour                              8760 non-null   int32                        
 8   month                             8760

In [ ]:
df_test_full7.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3957 entries, 0 to 3956
Data columns (total 17 columns):
 #   Column                            Non-Null Count  Dtype                        
---  ------                            --------------  -----                        
 0   load[MW]                          3957 non-null   float64                      
 1   DateCET/CEST                      3957 non-null   datetime64[ns, Europe/Warsaw]
 2   price_energy[EUR/MWh]             3957 non-null   float64                      
 3   index                             3957 non-null   object                       
 4   Photovoltaics generation [MWh]    3957 non-null   float64                      
 5   Wind generation [MWh]             3957 non-null   float64                      
 6   day                               3957 non-null   int32                        
 7   hour                              3957 non-null   int32                        
 8   month                             3957

In [ ]:
df_train_full7 = df_train_full7.drop(columns=['index', 'day', 'hour', 'month', 'year'])
df_test_full7 = df_test_full7.drop(columns=['index', 'day', 'hour', 'month', 'year'])

In [ ]:
df_train_full7.columns

Index(['load[MW]', 'DateCET/CEST', 'price_energy[EUR/MWh]',
       'Photovoltaics generation [MWh]', 'Wind generation [MWh]',
       'price_gas[EUR/m3]', 'price_carbon_permits[EUR]', 'price_coal',
       'temperature_2m (°C)', 'rain (mm)', 'wind_speed_100m (km/h)',
       'diffuse_radiation_instant (W/m?)'],
      dtype='object')

In [ ]:
df_train_full7 = df_train_full7.rename(columns={'load[MW]':'load_energy[MW]',
       'Photovoltaics generation [MWh]':'pv[MWh]', 'Wind generation [MWh]':'Wind[Mwh]', 'price_coal':'price_coal[EUR]',
       'temperature_2m (°C)':'temperature_2m[°C]', 'rain (mm)':'rain[mm]', 'wind_speed_100m (km/h)':'wind_speed_100m[km/h]',
       'diffuse_radiation_instant (W/m?)':'diffuse_radiation_instant[W/m?]'})


In [ ]:
df_test_full7 = df_test_full7.rename(columns={'load[MW]':'load_energy[MW]',
       'Photovoltaics generation [MWh]':'pv_energy_generation[MWh]', 'Wind generation [MWh]':'Wind_energy_generation[MWh]', 'price_coal':'price_coal[EUR]',
       'temperature_2m (°C)':'temperature_2m[°C]', 'rain (mm)':'rain[mm]', 'wind_speed_100m (km/h)':'wind_speed_100m[km/h]',
       'diffuse_radiation_instant (W/m?)':'diffuse_radiation_instant[W/m2]'})

In [ ]:
df_train_full7.to_csv('dataset_train.csv')
df_test_full7.to_csv('dataset_test.csv')